# Basic Settings

In [1]:
### 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 Runtime > Restart session 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (5,947 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 123594 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy
import scipy.stats as stats

# The Movies Dataset DDA

## Dataset
1. links.csv
2. movies_metadata.csv
3. keywords.csv
4. ratings.csv

### Dataset 불러오기

In [4]:
# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# data 불러오기

# CSV 파일 경로 지정
file_path01 = '/content/drive/MyDrive/SeSac 데이터 AI개발과정 mini project/dataset/archive/csv/links.csv'
file_path02 = '/content/drive/MyDrive/SeSac 데이터 AI개발과정 mini project/dataset/archive/csv/movies_metadata.csv'
file_path03 = '/content/drive/MyDrive/SeSac 데이터 AI개발과정 mini project/dataset/archive/csv/keywords.csv'
file_path04 = '/content/drive/MyDrive/SeSac 데이터 AI개발과정 mini project/dataset/archive/csv/ratings.csv'

# CSV 파일을 DataFrame으로 읽어오기
df_links = pd.read_csv(file_path01)
df_movies_metadata = pd.read_csv(file_path02)
df_keywords = pd.read_csv(file_path03)
df_ratings = pd.read_csv(file_path04)

<ipython-input-5-c5b98639a3d4>:11: DtypeWarning: Columns (0,1,3,8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies_metadata = pd.read_csv(file_path02)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---
### Dataset 정리

|번호|dataset명|설명|보유 column|비고|
|--|--|--|--|--|
|01|links.csv|- 다른 data table과의 연결<br> - 영화 id 매핑 테이블|moviesId, imdbId, tmdbId||
|02|movies_metadata.csv|영화관련 정보 data|adult, budget, homepage, id, imdb_id, original_language, original_title, overview, popularity, poster_path, release_date, revenue, runtime, status, tagline, title, video, vote_average, vote_count, belongs_to_collection_id, belongs_to_collection_name, belongs_to_collection_poster_path, belongs_to_collection_backdrop_path, genre_id, genre_name, production_company_id, production_company_name, production_country_iso, production_country_name, spoken_language_iso, spoken_language_name||
|03|keywords.csv|영화별 연관 keyword|id, keywords_id, keywords_name||
|04|ratings.csv|영화별 고객 평점 data|userId, movieId	rating, timestamp||


#### links.csv column정보

|Index|Variable|Definition|DataRange|Key|DataType|Comments|
|--|--|--|--|--|--|--|
|01|movieId|영화 ID|-|-|범주형|다른 table과의 매핑 작업에 활용|
|02|imdbId|IMDb에서 영화에 부여한 고유 ID|-|-|범주형|- 다른 table과의 매핑 작업에 활용<br>- 중복값 없음. str으로 타입 변경 후 사용 필요|
|03|tmdbId|TMDb에서 영화에 부여한 고유 ID|-|-|범주형|-다른 table과의 매핑 작업에 활용<br>- 중복값 존재. But, 다른 연결되는 table이 없어 삭제 필요(활용시 str 변환)|

- IMDb(Internet Movie Database) : 세계적으로 유명한 영화 데이터베이스로, 각 영화에 대해 고유한 식별자를 제공
- TMDb(The Movie Database) : 영화와 TV 프로그램에 대한 정보를 제공하는 데이터베이스, 별도의 식별자를 사용

In [7]:
df_links.head()

movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0

In [8]:
df_links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  45843 non-null  int64  
 1   imdbId   45843 non-null  int64  
 2   tmdbId   45624 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.0 MB


#### movies_metadata.csv column정보

|Index|Variable|Definition|DataRange|Key|DataType|Comments|
|--|--|--|--|--|--|--|
|01|adult|영화가 성인용인지 여부|-|True, False|범주형||
|02|budget|영화의 예산||숫자|수치형|영화 예산과 인기도의 상관관계 분석에 활용|
|03|homepage|영화 홈페이지 URL|-|URL|범주형|홈페이지가 존재하는 영화일수록 규모나 인기도가 클 것이라고 생각|
|04|id|영화 ID|-|-|범주형|links.csv table과의 매|
|05|imdb_id|IMDb에서 영화에 부여한 고유 ID|-|-|범주형|links.csv table과의 매핑|
|06|original_language|영화의 원어|-|ISO 639-1 코드(ISO에서 정의한 두 글자의 언어 코드)|범주형||
|07|original_title|영화의 원제|-|-|범주형||
|08|overview|영화 개요|-|-|-|text data|
|09|popularity|영화의 인기 지표|-|-|수치형(이산형)|인기 지표 계산 방법 확인 필요|
|10|poster_path|영화 포스터 이미지의 URL 경로|-|-|범주형|삭제 필요|
|11|release_date|영화 개봉일|-|datetime|수치형(연속형)|영화 개봉시기가 영화 흥행에 주는 영향 분석|
|12|revenue|영화 수익|-|-|수치형(이산형)|수익 예측 모델에 활용|
|13|runtime|영화 상영 시간|-|-|수치형(연속형)|영화 장르별 흥행 여부 예측 요소로 활용|
|14|status|영화 상태|-|영화의 제작 계획, 개봉, 제작 취소 등|범주형||
|15|tagline|영화 태그 라인|-|-|범주형|영화 태그라인 유무에 따른 흥행 비교<br>영화 정보에 따른 유행성 있는 태그라인 생성 모델 제작|
|16|title|영화 제목|-|-|범주형||
|17|video|영화의 비디오 여부|-|True, False|범주형|비디오로 제작된 영화의 경우 더욱 흥행하였을 것이다(가설 검증)|
|18|vote_average|영화 평균 평점|0.0 ~ 10.0||수치형(연속형)|영화 흥행에 대한 지표로 활용|
|19|vote_count|영화 평점수|0~최대|-|수치형(연속형)|영화 흥행에 대한 지표로 활용|
|20|belongs_to_collection_id|영화가 속한 컬렉션 ID|-|-|범주형|시리즈영화 대비 시리즈영화가 아닌 영화의 흥행 차이 등의 분석에 용이|
|21|belongs_to_collection_name|영화가 속한 컬렉션 이름|-|-|범주형|시리즈영화 대비 시리즈영화가 아닌 영화의 흥행 차이 등의 분석에 용이|
|22|belongs_to_collection_poster_path|영화가 속한 컬렉션의 포스터 경로|-|URL|범주형|포스터 경로는 필요 없음|
|23|belongs_to_collection_backdrop_path|영화가 속한 컬렉션의 배경 경로|-|URL|범주형|배경은 필요 없음|
|24|genre_id|영화 장르 ID|-|-|범주형|장르별 분석에 용이|
|25|genre_name|영화 장르 이름|-|-|범주형|장르별 분석에 용이|
|26|production_company_id|제작사 ID|-|-|범주형|분석 인사이트 제안 타겟이 제작사일 경우 필요|
|27|production_company_name|제작사 이름|-|-|범주형|분석 인사이트 제안 타겟이 제작사일 경우 필요|
|28|production_country_iso|제작 국가 ISO 코드|-|-|범주형|제작 국가별 주 제작 영화장르가 존재할 것이다(가설 검증)|
|29|production_country_name|제작 국가 이름|-|-|범주형|제작 국가별 주 제작 영화장르가 존재할 것이다(가설 검증)|
|30|spoken_language_iso|사용된 언어 ISO 코드|-|ISO 코드|범주형|영화의 원어와 차이가 무엇인지 확인 필요|
|31|spoken_language_name|사용된 언어 이름|-|-|범주형|영화의 원어와 차이가 무엇인지 확인 필요|


In [9]:
df_movies_metadata.head()

adult    budget                              homepage    id    imdb_id  \
0  False  30000000  http://toystory.disney.com/toy-story   862  tt0114709   
1  False  30000000  http://toystory.disney.com/toy-story   862  tt0114709   
2  False  30000000  http://toystory.disney.com/toy-story   862  tt0114709   
3  False  65000000                                   NaN  8844  tt0113497   
4  False  65000000                                   NaN  8844  tt0113497   

  original_language original_title  \
0                en      Toy Story   
1                en      Toy Story   
2                en      Toy Story   
3                en        Jumanji   
4                en        Jumanji   

                                            overview popularity  \
0  Led by Woody, Andy's toys live happily in his ...  21.946943   
1  Led by Woody, Andy's toys live happily in his ...  21.946943   
2  Led by Woody, Andy's toys live happily in his ...  21.946943   
3  When siblings Judy and Peter discover an encha...  17.015539   
4  When siblings Judy and Peter discover an encha...  17.015539   

                        poster_path  ... belongs_to_collection_poster_path  \
0  /rhIRbceoE9lR4veEXuwCC2wARtG.jpg  ...  /7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg   
1  /rhIRbceoE9lR4veEXuwCC2wARtG.jpg  ...  /7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg   
2  /rhIRbceoE9lR4veEXuwCC2wARtG.jpg  ...  /7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg   
3  /vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg  ...                               NaN   
4  /vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg  ...                               NaN   

   belongs_to_collection_backdrop_path  genre_id genre_name  \
0     /9FBwqcd9IRruEDUrTdcaafOMKUq.jpg      16.0  Animation   
1     /9FBwqcd9IRruEDUrTdcaafOMKUq.jpg      35.0     Comedy   
2     /9FBwqcd9IRruEDUrTdcaafOMKUq.jpg   10751.0     Family   
3                                  NaN      12.0  Adventure   
4                                  NaN      12.0  Adventure   

  production_company_id  production_company_name production_country_iso  \
0                   3.0  Pixar Animation Studios                     US   
1                   3.0  Pixar Animation Studios                     US   
2                   3.0  Pixar Animation Studios                     US   
3                 559.0         TriStar Pictures                     US   
4                 559.0         TriStar Pictures                     US   

    production_country_name  spoken_language_iso  spoken_language_name  
0  United States of America                   en               English  
1  United States of America                   en               English  
2  United States of America                   en               English  
3  United States of America                   en               English  
4  United States of America                   fr              Français  

[5 rows x 31 columns]

In [10]:
df_movies_metadata.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'belongs_to_collection_id',
       'belongs_to_collection_name', 'belongs_to_collection_poster_path',
       'belongs_to_collection_backdrop_path', 'genre_id', 'genre_name',
       'production_company_id', 'production_company_name',
       'production_country_iso', 'production_country_name',
       'spoken_language_iso', 'spoken_language_name'],
      dtype='object')

In [11]:
df_movies_metadata['status']

0         Released
1         Released
2         Released
3         Released
4         Released
            ...   
419774    Released
419775    Released
419776    Released
419777    Released
419778    Released
Name: status, Length: 419779, dtype: object

In [12]:
df_movies_metadata['popularity'].value_counts().sort_values()

popularity
0.163015        1
0.005577        1
0.110304        1
1.686871        1
0.018925        1
            ...  
6.752524     1152
3.463550     1400
5.051100     1710
3.861969     1800
0.958683    24700
Name: count, Length: 44069, dtype: int64

In [13]:
df_movies_metadata['budget'].value_counts()

budget
0           163805
0            65580
2000000      22237
30000000      5052
25000000      4689
             ...  
35               1
1552334          1
230              1
717000           1
2100             1
Name: count, Length: 1515, dtype: int64

In [14]:
df_movies_metadata['spoken_language_name']

0          English
1          English
2          English
3          English
4         Français
            ...   
419774     English
419775     English
419776     English
419777         NaN
419778     English
Name: spoken_language_name, Length: 419779, dtype: object

#### keywords.csv column정보

|Index|Variable|Definition|DataRange|Key|DataType|Comments|
|--|--|--|--|--|--|--|
|01|id|영화 ID|-|-|범주형|사용자 고유 식별 ID, str으로 변환후 활용 필요|
|02|keywords_id|영화에 대한 키워드 ID|-|-|범주형|str 변환 필요|
|03|keywords_name|키워드 이름|-|-|범주형|키워드 이름을 기준으로 비슷한 요소들끼리 분류모델을 만들 수 있을듯|


In [15]:
df_keywords.head()

id  keywords_id keywords_name
0  862        931.0      jealousy
1  862       4290.0           toy
2  862       5202.0           boy
3  862       6054.0    friendship
4  862       9713.0       friends

#### ratings.csv column정보

|Index|Variable|Definition|DataRange|Key|DataType|Comments|
|--|--|--|--|--|--|--|
|01|userId|사용자 ID|-|-|범주형|사용자 고유 식별 ID, str으로 변환후 활용 필요|
|02|movieId|영화 ID|-|-|범주형|links.csv table과 매핑하여 영화정보 확인 가능|
|03|rating|사용자가 부여한 평점|0-5|0.1, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0|범주형(순서형)|float로 되어있어서 계산 활용 가능|
|04|timestamp|평점이 매겨진 시간(UNIX 시간)|||||

In [16]:
df_ratings.head()

userId  movieId  rating   timestamp
0       1      110     1.0  1425941529
1       1      147     4.5  1425942435
2       1      858     5.0  1425941523
3       1     1221     5.0  1425941546
4       1     1246     5.0  1425941556

In [17]:
df_ratings['rating'].value_counts()

rating
4.0    6998802
3.0    5256722
5.0    3812499
3.5    3116213
4.5    2170441
2.0    1762440
2.5    1255358
1.0     843310
0.5     404897
1.5     403607
Name: count, dtype: int64

In [18]:
df_ratings['timestamp'].value_counts()

timestamp
825638400     685
1485450595    280
1462647814    264
1490758295    233
1469059264    223
             ... 
1129026458      1
1129028829      1
1129027512      1
1129029528      1
1257031858      1
Name: count, Length: 20549435, dtype: int64

---

## 분석 주제 아이디어
<모델 제작>
- 영화 수익 예측 모델(제작국가, 제작 예산 규모, 장르 등의 요인을 입력하여 수익 예측하는 모델 만들기 + casting 정보도 활용 가능하면 좋을 듯)
- 영화 흥행 여부 예측 모델(제작국가, 제작 예산 규모, 장르 등의 요인을 입력하여 흥행 여부 예측 모델 만들기 + casting 정보도 활용 가능하면 좋을 듯)
- 영화 평점 예측 모델(예산, 상영시간, 장르, 국가, 인기도 등의 요인을 입력하여 회귀모델 제작)
- 영화 개요를 활용하여 장르 분류 모델 제작
- 영화 정보에 따른 유행성 있는 태그라인 생성 모델 제작
- 영화 추천 시스템

<분석 주제>
- 영화 인기도에 영향을 주는 요인을 분석 및 인사이트 도출
- 시리즈영화 대비 시리즈영화가 아닌 영화의 흥행 차이 분석
- 장르별 수익성 분석(장르별 수익성과 인기도를 분석하여 어떤 장르가 수익성이 높은지 파악) > 영화 투자사 입장에서의 인사이트
- 개봉시기와 흥행 성적의 상관관계 분석(영화 개봉 시기와 흥행 성적의 상관관계 분석)
- 제작사별 성공 패턴 분석(제작사별로 성공적인 영화의 패턴을 분석하여 어떤 제작사가 어떤 유형의 영화에서 성공적인지 파악) > 제작사별로 강점을 분석하여 제작사 선정 인사이트 도출

---
- 영화 포스터 > 이미지 딥러닝?